In [1]:
import numpy as np
import time

In [2]:
n = 16

In [3]:
def getseed(n = 16):
    """
    Generates the seed for the Gibbs sampler.
    Inputs:
    - n: (int) number of neurons in the network
    """
    return np.random.randint(2, size = 16)   

def getW(n = 16):
    """
    Generates the matrix to learn using MPF.
    Inputs:
    - n: (int) number of neurons in the network
    """
    U = np.random.normal(0, 1, (n, n))
    W = 0.5 * (U + U.T)
    np.fill_diagonal(W, 0)
    np.save('wildW.dat', W)
    return W

def getb(n = 16):
    """
    Generates the bias to learn using MPF.
    Inputs:
    - n: (int) number of neurons in the network
    """
    b = np.random.randint(2, size = 16)
    np.save('wildb.dat', b)
    return b   


def sigmoid(x):
    """
    Takes in a vector x and returns its sigmoid activation.
    Input:
    - x: a numpy array
    """
    return 1/(1 + np.exp(-x))

In [26]:
seed = getseed()
W = getW()
b = getb()

In [5]:
def one_state_update(x, W, b, s):
    """
    Does a single update of the sth neuron of the network.
    Inputs:
    - x: current state of the network to produce a new state
    - W: numpy array of weights
    - b: numpy array of biases
    """
#     print ('Old:', x, id(x))
    p = sigmoid(np.dot(W, x) + b)
    new_x = np.copy(x)
#     print (id(new_x))
    new_x[s] = np.random.binomial(1, p[s], 1) 
#     print ('Probabilities:', p[5])
#     print ('Old:', x, id(x))
#     print ('New:', new_x, id(new_x))
    return new_x

In [7]:
def burn_in(x, W, b, n = 10000):
    """
    Performs the burning in before doing the Gibbs sampling.
    """
    v = x.shape[0]
    for i in range(n * v):
        s = np.random.randint(0, v)
        x = one_state_update(x, W, b, s)
    return x    

In [11]:
def n_updates(x, W, b, n = 100):
    """
    Performs n times of the one_state_update.
    Inputs:
    - x: current state of the network to produce a new state
    - W: numpy array of weights
    - b: numpy array of biases
    - n: (int) number of updates to be made
    """
    v = x.shape[0]
    for i in range(n * v):
        s = np.random.randint(0, v)
        x = one_state_update(x, W, b, s)
    return x

In [45]:
def mixing(x, W, b, n = 50000, m = 100, savesamples = 'True'):
    """
    Does mixing for m times before obtaining a single sample.
    Inputs:
    - x: current state of the network to produce a new state
    - W: numpy array of weights
    - b: numpy array of biases
    - n: (int) number of samples to be generated
    - m: (int) number of updates before a sample is saved
    """
    samples = np.zeros((n, x.shape[0]))
    for i in range(n):
        s = n_updates(x, W, b, m)
        samples[i, :] = s
        
    if savesamples == "True":
        np.save('gibbs-sample.dat', samples)
        print ('Samples are saved as "gibbs-sample.dat.npy"')
    elif savesamples == "False":
        print ('Samples were not saved. Run np.save("gibbs-sample.dat", sample) to save them. ')
    else:
        raise ValueError("savesamples must be 'True' or 'False'")    
    return samples

In [43]:
def sampling(x, W, b, n = 50000, m = 100, savesamples = 'True'):
    tic = time.time()
    print ('Burning in...')
    burnt = burn_in(x, W, b)
    print ('Mixing...')
    samples = mixing(x, W, b, n, m)
    toc = time.time()
    print ('Time taken to create %d samples is %f2 minutes' % (n, (toc - tic)/60.))
    return samples

In [ ]:
sampling(seed, W, b)

Burning in...
Mixing...


In [42]:
print (seed)
print (W)
print (b)

[1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0]
[[ 0.         -0.81060884 -0.54606147 -0.67406265 -0.85362585 -0.20510847
   0.4568468  -0.50514437  0.09721725 -0.18397757  0.37526607 -1.7303635
   0.91895714  0.01726934 -0.26658173 -0.2881471 ]
 [-0.81060884  0.         -0.4649697  -0.85263503 -0.17584544  0.64309694
   0.47892279 -0.54006362  0.93707694  0.01920955 -0.19812411 -0.40039792
   1.95167486 -0.91055876  0.72286565 -0.63941877]
 [-0.54606147 -0.4649697   0.          0.23200601 -0.15758482  0.81953274
  -0.23370028 -0.62390941  0.15258657  0.11864316  0.81503253  0.82579841
  -1.08052015 -0.04188282 -0.02085416  1.44501935]
 [-0.67406265 -0.85263503  0.23200601  0.          0.75349221 -0.99533129
   1.88049937 -0.68285743  0.21500688  1.23018959  0.15623693  0.06925602
   0.30097856  1.6129382  -0.40201307 -0.2248305 ]
 [-0.85362585 -0.17584544 -0.15758482  0.75349221  0.          0.01046808
   0.60427758  0.49901599  1.06101995  0.2698569   1.17352421  1.25997474
   0.13650027  1.096908